# Chapter 18: with, match, and else Blocks

## Context Managers and with Blocks

In [8]:
# tag::MIRROR_EX[]
import sys

class LookingGlass:

    def __enter__(self):  # <1>
        self.original_write = sys.stdout.write  # <2>
        sys.stdout.write = self.reverse_write  # <3>
        return 'JABBERWOCKY'  # <4>

    def reverse_write(self, text):  # <5>
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):  # <6>
        sys.stdout.write = self.original_write  # <7>
        if exc_type is ZeroDivisionError:  # <8>
            print('Please DO NOT divide by zero!')
            return True  # <9>
        # <10>
# end::MIRROR_EX[]

1. Python invokes `__enter__` with on argument besides `self`.
2. Hold the original `sys.stdout.write` method, so we can restore it later.
3. Monkey-patch `sys.stdout.write` with our own function.
4. Return the 'JABBERWOCKY' string just so we have something to put in the target variable `what`.
5. Our replacement to `sys.stdout.write` reverse the `text` argument and calls the original `sys.stdout.write` method.
6. Python calls `__exit__` with `None`, `None`, `None` if all went well; if an exception is raised, the three arguments get the exception data, as described after this example.
7. Restore the original `sys.stdout.write` method.
8. If the exception is not `None` and its type is `ZeroDivisionError`, print a message.
9. Return `True` to indicate that the exception was handled.
10. If `__exit__` returns `None` or any falsy value, any exception raised in the with block will be propagated.

The three argument passed to `__exit__` are:
- `exc_type`: The exception class (e.g., `ZeroDivisionError`).
- `exc_value`: The exception instance. Sometimes, parameters passed to the exception constructor--such as the error message--can be found in `exc_value.args`.
- `traceback`: A traceback object.

In [10]:
with LookingGlass() as what: #1
    print('Alice, Kitty and Snowdrop') #2
    print(what)
    print(5/0)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
Please DO NOT divide by zero!


In [5]:
what #3

'JABBERWOCKY'

In [6]:
print('Back to normal.') #4

Back to normal.


1. The context manager is an instance of `LookingGlass`； Python calls `__enter__` on the context manager and the result is bound to `what`.
2. Print a `str`, then the value of the target variable `what`. The output of each `print` will come out reversed.
3. Now the with block is over. We can see that the value of `what` is back to normal.
4. Program output is no longer reversed.

### The contextlib Utilities

The `contextlib` package also includes:

- `closing`: A function to build context managers out of objects that provide a `close()` method but don't implement the `__enter__/__exit__` protocol.
- `suppress`: A context manager to temporarily ignore specified exceptions.
- `nullcontext`: A context manager that does nothing, to simplify conditional logic around objects that may not implement a suitable context manager. It serves as a stand-in when conditional code before the `with` block may or may not provide a context manager for the with statement.


### Using @contextmanager

The `@contextmanager` decorator is an elegant and practical tool that brings together three distinctive Python features: a function decorator, a generator, and the `with` statement.

In [11]:
import contextlib
import sys

@contextlib.contextmanager #1
def looking_glass():
    original_write = sys.stdout.write #2
    
    def reverse_write(text): #3
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write #4
    yield 'JABBERWOCKY' #5
    sys.stdout.write = original_write #6

Test-driving the `looking_glass` context manager function.

In [12]:
with looking_glass() as what: #1
    print('Alice, Kitty and Snowdrop') #2
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [13]:
what

'JABBERWOCKY'

In [14]:
print('Back to normal.')

Back to normal.


A little-known feature of `@contextmanager` is that the generators decorated with it can also be used as decorators themselves. That happens because `@contextmanager` is implemented with the `contextlib.ContextDecorator` class.

In [15]:
@looking_glass()
def verse():
    print('The time has come')    
    
verse()

emoc sah emit ehT


In [16]:
print('Back to normal.')

Back to normal.


A context manager for rewriting files in place. [inplace source code](https://www.zopatista.com/python/2013/11/26/inplace-file-rewriting/)

In [ ]:
from contextlib import contextmanager
import io
import os


@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.

    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'):
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                        encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                            encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

In [ ]:
import csv

with inplace(csvfilename, 'r', newline='') as (infh, outfn):
    reader = csv.reader(infh)
    writer = csv.writer(outfh)
    
    for row in reader:
        row += ['new', 'columns']
        writer.writerow(row)